In [2]:
import pandas as pd

# Load CSV files
orders = pd.read_csv("data/orders_with_buckets.csv")
order_details = pd.read_csv("data/order_details.csv")

# Merge orders with order details
merged = order_details.merge(orders, on="order_id")

# Pivot table: index = date + time_bucket, columns = pizza_id, values = sum(quantity)
pivot = merged.pivot_table(
    index=['date', 'time_bucket'],
    columns='pizza_id',
    values='quantity',
    aggfunc='sum',
    fill_value=0  # sparse encoding: 0 if not ordered
).reset_index()

# Optional: flatten column names
pivot.columns.name = None

# Save to CSV
pivot.to_csv("data/daily_timebucket_pizza_sales.csv", index=False)

print("Merged CSV saved as 'daily_timebucket_pizza_sales.csv'")


Merged CSV saved as 'daily_timebucket_pizza_sales.csv'


In [14]:
import pandas as pd
import holidays

# Load the oil price CSV
oil = pd.read_csv("data/oil_prices.csv")

# Convert date column
oil['Date'] = pd.to_datetime(oil['Date'], infer_datetime_format=True, errors='coerce')
oil = oil.dropna(subset=['Date'])

# Keep only 2015
oil_2015 = oil[oil['Date'].dt.year == 2015].copy()

# Rename for convenience
oil_2015 = oil_2015.rename(columns={'Europe Brent Spot Price FOB (Dollars per Barrel)': 'oil_price'})

# Set Date as index
oil_2015 = oil_2015.set_index('Date')

# Create full date range for 2015
full_dates = pd.date_range(start='2015-01-01', end='2015-12-31')

# Reindex to include all dates
oil_2015 = oil_2015.reindex(full_dates)

# Fill missing oil prices with mean
mean_price = oil_2015['oil_price'].mean()
oil_2015['oil_price'] = oil_2015['oil_price'].fillna(mean_price)

# Reset index and format date as DD/MM/YYYY
oil_2015 = oil_2015.reset_index().rename(columns={'index': 'date'})
oil_2015['date'] = oil_2015['date'].dt.strftime('%d/%m/%Y')

# Add holiday flag
us_holidays = holidays.US(years=2015)
oil_2015['is_holiday'] = pd.to_datetime(oil_2015['date'], format='%d/%m/%Y').dt.date.apply(lambda x: 1 if x in us_holidays else 0)

oil_2015['oil_price'] = oil_2015['oil_price'].round(2)

# Save final CSV
oil_2015.to_csv("data/oil_2015.csv", index=False)

print(oil_2015.head(10))


         date  oil_price  is_holiday
0  01/01/2015      52.32           1
1  02/01/2015      55.38           0
2  03/01/2015      52.32           0
3  04/01/2015      52.32           0
4  05/01/2015      51.08           0
5  06/01/2015      50.12           0
6  07/01/2015      49.06           0
7  08/01/2015      49.43           0
8  09/01/2015      47.64           0
9  10/01/2015      52.32           0


/var/folders/xl/t0lmyr3n2zj7_lvsrc1d1bv40000gn/T/ipykernel_56483/2954857212.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  oil['Date'] = pd.to_datetime(oil['Date'], infer_datetime_format=True, errors='coerce')


In [15]:
import pandas as pd

# Load pizza sales pivot table
pizza = pd.read_csv("data/daily_timebucket_pizza_sales.csv")
pizza['date'] = pd.to_datetime(pizza['date'], format="%d/%m/%Y")

# Load oil prices CSV
oil = pd.read_csv("data/oil_2015.csv")
oil['date'] = pd.to_datetime(oil['date'], format="%d/%m/%Y")

# Merge on date
merged = pizza.merge(oil, on='date', how='left')

# Optional: keep date in DD/MM/YYYY format for final CSV
merged['date'] = merged['date'].dt.strftime('%d/%m/%Y')

# Save merged CSV
merged.to_csv("data/daily_sales.csv", index=False)

print("Merged CSV saved as 'data/daily_sales.csv'")
print(merged.head())


Merged CSV saved as 'data/daily_sales.csv'
         date time_bucket  bbq_ckn_l  bbq_ckn_m  bbq_ckn_s  big_meat_s  \
0  01/01/2015   Afternoon          2          1          0           2   
1  01/01/2015      Dinner          2          2          0           1   
2  01/01/2015       Lunch          2          1          1           2   
3  01/02/2015   Afternoon          2          1          0           2   
4  01/02/2015      Dinner          2          5          1           2   

   brie_carre_s  calabrese_l  calabrese_m  calabrese_s  ...  the_greek_l  \
0             0            0            0            0  ...            0   
1             0            0            0            0  ...            0   
2             0            0            1            0  ...            0   
3             0            1            3            0  ...            1   
4             1            0            1            0  ...            0   

   the_greek_m  the_greek_s  the_greek_xl  the_greek_xx